In [1]:
"""
Balance Tracker - revised
Developed by CD
[v?]
"""
from pathlib import Path

import pandas as pd # type: ignore
import openpyxl # type: ignore
from openpyxl.utils import get_column_letter # type: ignore

import src.balance_tracker_pipeline_v2
import src.cdutils.database.fdic_recon

# Fetch Data from COCC
data_prior = src.cdutils.database.fdic_recon.fetch_data_dec24()
data_current = src.cdutils.database.fdic_recon.fetch_data_jan25()

_, data_prior_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_prior)
_, data_current_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_current)



In [2]:
data_current_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Category             6 non-null      object 
 1   Net Balance          6 non-null      float64
 2   Net Balance Rounded  6 non-null      float64
dtypes: float64(2), object(1)
memory usage: 276.0+ bytes


In [3]:
data_prior_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Category             6 non-null      object 
 1   Net Balance          6 non-null      float64
 2   Net Balance Rounded  6 non-null      float64
dtypes: float64(2), object(1)
memory usage: 276.0+ bytes


In [24]:
def merging_summary_tables(data_prior_summary: pd.DataFrame, data_current_summary: pd.DataFrame) -> pd.DataFrame:
    """
    Combining the two summary tables to find the monthly delta for each category
    """
    df = pd.merge(data_prior_summary, data_current_summary, how='inner', on='Category', suffixes=('_prior','_current'))
    return df

In [25]:
df = merging_summary_tables(data_prior_summary, data_current_summary)

In [26]:
df

,Category,Net Balance_prior,Net Balance Rounded_prior,Net Balance_current,Net Balance Rounded_current
0,C&I,1.167990e+08,116799.0,1.166215e+08,116622.0
1,CRE,1.108016e+09,1108016.0,1.112945e+09,1112945.0
2,Consumer,7.703546e+06,7704.0,7.474293e+06,7474.0
3,HOA,1.521513e+07,15215.0,1.712787e+07,17128.0
4,Indirect,2.879786e+08,287979.0,2.805174e+08,280517.0
5,Residential,9.192768e+08,919277.0,9.210364e+08,921036.0


In [27]:
df['Delta'] = (df['Net Balance_current'] - df['Net Balance_prior']) / 1000


In [28]:
df['Delta'].sum()

np.float64(733.0998600001062)

In [29]:
custom_order = ['CRE','C&I','HOA','Residential','Consumer','Indirect']
df['Category'] = pd.Categorical(df['Category'], categories=custom_order, ordered=True)
df = df.sort_values('Category')
df = df[['Category','Delta']].copy()
df

,Category,Delta
1,CRE,4928.64188
0,C&I,-177.46989
3,HOA,1912.74254
5,Residential,1759.61187
2,Consumer,-229.25299
4,Indirect,-7461.17355


In [33]:
TEMPLATE_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
# OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD_test.xlsx')

def update_excel_template(template_path, df, output_path):
    wb = openpyxl.load_workbook(template_path)
    sheet = wb["Calendar 2025"]

    category_to_row = {}
    for row in sheet.iter_rows(min_col=1, max_col=1):
        cell = row[0]
        if cell.value is not None:
            category_to_row[cell.value] = cell.row

    for index, record in df.iterrows():
        category = record['Category']
        delta = record['Delta']

        if category not in category_to_row:
            print(f"Warning: Category '{category}' not in column A. Skipping.")
            continue

        row_number = category_to_row[category]
        col_number = 3

        while col_number <= 14: # Stop after December written
            current_cell = sheet.cell(row=row_number, column=col_number)
            if current_cell.value is None:
                current_cell.value = delta
                break
            col_number += 1
        else:
            print(f"Error, Calendar completed. Please refresh for a new year in a new template tab.")

    
    wb.save(output_path)
    wb.close()


In [47]:
update_excel_template(TEMPLATE_PATH, df, OUTPUT_PATH)


In [32]:
df['Delta'].sum()

np.float64(733.0998600001058)

In [4]:
"""
Fetching data module. Aim is import all necessary fields up front, but if needed, you can define another function to be called here.

This uses a sliding window for the trailing 2 months, current and prior.

Usage:
    import src.cdutils.database
"""
import datetime
from typing import Dict

from sqlalchemy import text # type: ignore

import src.cdutils.database.connect

"""
Fetch data from COCC in a sliding window fashion (trailing month end dates)
"""
effdates = text("""
SELECT DISTINCT
    a.EFFDATE
FROM 
    COCCDM.WH_ACCTCOMMON a
WHERE
    MONTHENDYN = 'Y'
ORDER BY EFFDATE DESC
""")

queries = [
    {'key':'effdates', 'sql':effdates, 'engine':2},
]

data = src.cdutils.database.connect.retrieve_data(queries)

effdates = data['effdates'].copy()

recent_me = effdates['effdate'][0]
prior_me = effdates['effdate'][1]


In [5]:
recent_me

Timestamp('2025-01-31 00:00:00')

In [6]:
prior_me

Timestamp('2024-12-31 00:00:00')

In [ ]:

    def main_query(monthend_date: datetime) -> Dict:
        """
        Takes in a date to query on and returns a dictionary with dataframes for each table.
        """
        wh_acctcommon = text(f"""
        SELECT
            a.EFFDATE,
            a.ACCTNBR,
            a.OWNERSORTNAME,
            a.PRODUCT,
            a.NOTEOPENAMT,
            a.RATETYPCD,
            a.MJACCTTYPCD,
            a.CURRMIACCTTYPCD,
            a.CURRACCTSTATCD,
            a.NOTEINTRATE,
            a.BOOKBALANCE,
            a.NOTEBAL
        FROM
            COCCDM.WH_ACCTCOMMON a
        WHERE
            (a.CURRACCTSTATCD IN ('ACT','NPFM')) AND
            (a.EFFDATE = TO_DATE('{monthend_date}', 'yyyy-mm-dd hh24:mi:ss'))
        """)

        wh_loans = text("""
        SELECT
            a.ACCTNBR,
            a.ORIGDATE,
            a.CURRTERM,
            a.LOANIDX,
            a.RCF,
            a.AVAILBALAMT,
            a.FDICCATDESC,
            a.ORIGBAL
        FROM
            COCCDM.WH_LOANS a
        WHERE
            (a.RUNDATE = TO_DATE('{monthend_date}', 'yyyy-mm-dd hh24:mi:ss'))
        """)

        wh_acctloan = text(f"""
        SELECT
            a.ACCTNBR,
            a.CREDITLIMITAMT,
            a.ORIGINTRATE,
            a.MARGINFIXED,
            a.FDICCATCD,
            a.AMORTTERM,
            a.TOTALPCTSOLD,
            a.COBAL,
            a.CREDLIMITCLATRESAMT
        FROM
            COCCDM.WH_ACCTLOAN a
        WHERE
            (a.EFFDATE = TO_DATE('{monthend_date}', 'yyyy-mm-dd hh24:mi:ss'))
        """)

        wh_acct = text("""
        SELECT
            a.ACCTNBR,
            a.DATEMAT
        FROM
            COCCDM.WH_ACCT a
        WHERE
            (a.RUNDATE = TO_DATE('{monthend_date}', 'yyyy-mm-dd hh24:mi:ss'))
        """)

        queries = [
            {'key':'wh_acctcommon', 'sql':wh_acctcommon, 'engine':2},
            {'key':'wh_loans', 'sql':wh_loans, 'engine':2},
            {'key':'wh_acctloan', 'sql':wh_acctloan, 'engine':2},
            {'key':'wh_acct', 'sql':wh_acct, 'engine':2},
        ]

        data = src.cdutils.database.connect.retrieve_data(queries)
        return data

    prior_data = main_query(prior_me)
    current_data = main_query(recent_me)

    return prior_data, current_data 

